## Extracting Image Feature using MobileNetV1
preprocessing image sekalian dilakuin disini 

### Import Library

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import mobilenet
from tqdm import tqdm
import os
# Menonaktifkan pesan log TensorFlow yang kurang relevan
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'



### Load Dataset Train & Test
Load dari hasil split_data

In [ ]:
def load_split_data(split_dir, train_filename, test_filename):
    """Memuat dataset train dan test, dan mendeteksi rasio split dari nama file."""
    
    train_path = os.path.join(split_dir, train_filename)
    test_path = os.path.join(split_dir, test_filename)
    
    try:
        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)
        
        # Ekstrak angka dari nama file untuk mendapatkan rasio (contoh: '8020')
        split_ratio_str = re.search(r'(/d+)', train_filename).group(1)
        
        print(f"Data latih berhasil dimuat: {train_df.shape[0]} baris.")
        print(f"Data uji berhasil dimuat: {test_df.shape[0]} baris.")
        print(f"Rasio split terdeteksi: {split_ratio_str}")
        
        return train_df, test_df, split_ratio_str
        
    except FileNotFoundError as e:
        print(f"Error: Salah satu file tidak ditemukan. {e}")
        return None, None, None

# --- Konfigurasi dan Eksekusi (CUSTOM)---
SPLIT_DATA_DIR = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/split_data'
TRAIN_FILENAME = 'train8020.csv'  
TEST_FILENAME = 'test8020.csv'

train_df, test_df, split_ratio = load_split_data(SPLIT_DATA_DIR, TRAIN_FILENAME, TEST_FILENAME)

Data latih berhasil dimuat: 13044 baris.
Data uji berhasil dimuat: 3261 baris.
Rasio split terdeteksi: 8020


### Setup Model MobileNetV1
Konfigurasi model untuk ekstraksi

In [4]:
def load_mobilenet_model():
    """Memuat model MobileNetV1 pre-trained tanpa lapisan klasifikasi."""
    
    # Memuat model MobileNetV1 dengan bobot dari ImageNet -> pretrained imagenet
    # include_top=False berarti kita membuang lapisan 'fully connected' terakhir
    # pooling='avg' menambahkan lapisan Global Average Pooling untuk menghasilkan vektor fitur
    model = mobilenet.MobileNet(
        weights='imagenet', 
        include_top=False, 
        input_shape=(224, 224, 3),
        pooling='avg'
    )
    print("Model MobileNetV1 berhasil dimuat.")
    return model

# --- Eksekusi ---
feature_extractor_model = load_mobilenet_model()

# Menampilkan ringkasan arsitektur model
feature_extractor_model.summary()

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Model MobileNetV1 berhasil dimuat.


Model: "mobilenet_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 3,206,976 (12.23 MB)

 Non-trainable params: 21,888 (85.50 KB)

### Preprocessing Image 
Setup preprocessing image

In [13]:
def preprocess_and_extract_features(img_path, model):
    """
    Memproses satu gambar (resize, normalisasi) dan mengekstrak
    vektor fiturnya menggunakan model yang diberikan.
    
    Args:
        img_path (str): Path ke file gambar.
        model (tf.keras.Model): Model untuk ekstraksi fitur (misal: MobileNetV1).
        
    Returns:
        np.ndarray: Vektor fitur gambar, atau None jika gambar gagal diproses.
    """
    try:
        # 1. Muat gambar dan resize ke 224x224 piksel
        img = image.load_img(img_path, target_size=(224, 224))
        
        # 2. Ubah gambar menjadi array numpy
        img_array = image.img_to_array(img)
        
        # 3. Tambahkan dimensi batch (model Keras mengharapkan input batch)
        img_expanded = np.expand_dims(img_array, axis=0)
        
        # 4. Normalisasi piksel sesuai standar MobileNet (-1 sampai 1)
        img_preprocessed = mobilenet.preprocess_input(img_expanded)
        
        # 5. Ekstraksi fitur (tanpa menghitung gradien)
        features = model.predict(img_preprocessed, verbose=0)
        
        # 'Flatten' output untuk mendapatkan vektor 1D
        return features.flatten()
    
    except Exception as e:
        # Tangani jika file gambar corrupt atau tidak bisa dibuka
        # print(f"Gagal memproses {img_path}: {e}")
        return None

print("Fungsi 'preprocess_and_extract_features' siap digunakan.")

Fungsi 'preprocess_and_extract_features' siap digunakan.


### Melakukan Ekstraksi Fitur
ekstrak fitur gambar

In [14]:
def run_feature_extraction(df, model, description):
    """Menerapkan ekstraksi fitur ke seluruh DataFrame."""
    
    # Inisialisasi tqdm untuk pandas
    tqdm.pandas(desc=description)
    
    # Terapkan fungsi ke kolom 'image_path' dan tampilkan progress bar
    feature_vectors = df['image_path'].progress_apply(lambda path: preprocess_and_extract_features(path, model))
    
    return np.array(feature_vectors.tolist())

# --- Eksekusi untuk Data Latih dan Uji ---
print("Memulai ekstraksi fitur untuk data train...")
X_train_image = run_feature_extraction(train_df, feature_extractor_model, "Extracting Train Features")

print("/nMemulai ekstraksi fitur untuk data uji...")
X_test_image = run_feature_extraction(test_df, feature_extractor_model, "Extracting Test Features")

print("/nProses ekstraksi fitur selesai.")
print(f"Ukuran matriks fitur gambar train: {X_train_image.shape}")
print(f"Ukuran matriks fitur gambar test: {X_test_image.shape}")

Memulai ekstraksi fitur untuk data train...


Extracting Train Features:  12%|█▏        | 1543/13044 [03:55<22:18,  8.59it/s] c:/Users/LEGION/AppData/Local/Programs/Python/Python313/Lib/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Extracting Train Features: 100%|██████████| 13044/13044 [25:32<00:00,  8.51it/s]



Memulai ekstraksi fitur untuk data uji...


Extracting Test Features:  77%|███████▋  | 2500/3261 [04:53<01:28,  8.59it/s]c:/Users/LEGION/AppData/Local/Programs/Python/Python313/Lib/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Extracting Test Features: 100%|██████████| 3261/3261 [06:23<00:00,  8.50it/s]


Proses ekstraksi fitur selesai.
Ukuran matriks fitur gambar train: (13044, 1024)
Ukuran matriks fitur gambar test: (3261, 1024)


### Save File Ekstraksi
simpen hasil ekstraksi fitur gambar dari mobilenetv1

In [ ]:
def save_features(features, output_path):
    """Menyimpan matriks fitur ke dalam file .npy."""
    
    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        np.save(output_path, features)
        print(f"Fitur berhasil disimpan di: {output_path}")
    except Exception as e:
        print(f"Gagal menyimpan fitur. Error: {e}")

# --- Konfigurasi dan Eksekusi ---
IMAGE_ARTIFACTS_DIR = r'/home/spil/1Bagus/BACKUP/TA/Multimodal_Process_Exploration/DATA/_image_artifacts'
dynamic_folder_name = f'mobilenetv1_{split_ratio}'

# Simpan fitur train
save_features(X_train_image, os.path.join(IMAGE_ARTIFACTS_DIR, dynamic_folder_name, 'X_train_image.npy'))

# Simpan fitur test
save_features(X_test_image, os.path.join(IMAGE_ARTIFACTS_DIR, dynamic_folder_name, 'X_test_image.npy'))

Fitur berhasil disimpan di: E:/$7th/TA/Eksploring_TF-IDF/DATA/image_artifacts/mobilenetv1_8020/X_train_image.npy
Fitur berhasil disimpan di: E:/$7th/TA/Eksploring_TF-IDF/DATA/image_artifacts/mobilenetv1_8020/X_test_image.npy
